# This AutoML Boilerplate Contains the sample code for: 
1. Initiatilzing AI Platform 
1. Read Datasets 
1. Create Datasets 
1. Create the Job Training 
1. Create the Model 
1. Review Model Evaluations 
1. Create Endpoints 
1. Create Batch and Online Prediction Jobs  

# Initialize AI Platform 

```python
aiplatform.init(project=PROJECT_ID, location=REGION)
```

# Datasets

```python
#
# set variables 
# 
datasetTypeSelection = "ImageDataset" #@param ["ImageDataset", "TextDataset", "TabularDataset", "VideoDataset"]
datasetImportSchemaSelection = "image.bounding_box" #@param ["image.bounding_box", "text.single_label_classification", "video.object_tracking", "tabluarDeleteTheLine"]

##
# Proceed with dataset creation/validation
## 
# Create dataset if it doesn't exist

datasets = aiplatform.{datasetTypeSelection}.list(filter = f"display_name={dataset_display_name}")

if datasets:
    dataset = datasets[0]
    print(f"Dataset Exists: {datasets[0].display_name}")
else:
    dataset = aiplatform.{datasetTypeSelection}.create (
    display_name=dataset_display_name,
    gcs_source=dataset_source_uri,
    import_schema_uri= aiplatform.schema.dataset.ioformat.{datasetImportSchemaSelection},
    )
    print(f"Dataset Created: {dataset.display_name}")
    
print(f'Review the Dataset in the Console:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/datasets/{datasets[0].resource_name}?project={PROJECT_ID}')


```

## Read Datasets
 
### PD read data
```python

# Read the data
dataset_sample = pd.read_csv(dataset_source_public,nrows=5)
                                   
# Use the next code cell to print the first five rows of the data.
dataset_sample.head()

# Shape of training data (num_rows, num_columns)
print(dataset_sample.head())

```

### Cat Read data

```python
count = ! gsutil cat $dataset_source_uri | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $dataset_source_uri | head -10
```

# Training Jobs

```python

## set variables 

JobTypeSelection = "AutoMLImageTrainingJob" #@param ["AutoMLImageTrainingJob", "AutoMLTextTrainingJob", "AutoMLVideoTrainingJob", "AutoMLTabularTrainingJob"]
PredictionTypeSelection =  "classification" #@param ["classification", "object_detection"]
MultiLabelSelection= "False" #@param ["False", "True"]

## replace the variable with the text in the dropdown on the code directly

# Define the training job and create one if it doesn't exit 
jobs = aiplatform.AutoMLImageTrainingJob.list(filter = f"display_name={job_display_name}")

if jobs:
    job = jobs[0]
    print(f"Jobs Exists: {jobs[0].display_name}")
else:
    job = aiplatform.AutoMLImageTrainingJob(
    display_name=job_display_name,
    prediction_type="classification",
    multi_label=False,
    )
    print(f"job Created: {job.display_name}")
    
print(f'Review the job in the Console:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/jobs/{jobs[0].resource_name}?project={PROJECT_ID}')

```

# Models

```python

# Create model job if it doesn't exist
models = aiplatform.Model.list(filter = f"display_name={model_display_name}")

if models:
    model = models[0]
    print(f"Model Exists: {models[0].display_name}")
else:
    model = job.run(
    dataset=dataset,
    model_display_name=model_display_name,
    training_fraction_split=0.1,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    sync=True,
)
print(f"Model Created: {model.display_name}")


```

# Model Evaluations

```python

model_evaluations = model.list_model_evaluations(filter = f"display_name={model_display_name}")


for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())


# Get model resource ID
models = aiplatform.Model.list(filter = f"display_name={model_display_name}")

# Get a reference to the Model Service client
client_options = {"api_endpoint": endpoint_display_name}
model_service_client = aiplatform.gapic.ModelServiceClient(
    client_options=client_options
)

model_evaluations = model_service_client.list_model_evaluations(
    parent=models[0].resource_name
)
model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

```

# Endpoints

```python

### Create/Retrieve The Endpoint For This Series

# Check if Endpoint Exists 
endpoints = aiplatform.Endpoint.list(filter = f"display_name={endpoint_display_name}")

if endpoints:
    endpoint = endpoints[0]
    print(f"Endpoint Exists: {endpoints[0].display_name}")
else:
    endpoint = aiplatform.Endpoint.create(
        display_name = f"{endpoint_display_name}",
        labels = {'series' : f"{automl_type}"}    
    )
    print(f"Endpoint Created: {endpoint.display_name}")

print(f'Review the Endpoint in the Console:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/endpoints/{endpoint.name}?project={PROJECT_ID}')

```

# Jobs

```python
# Define the training job and create one if it doesn't exit 
jobs = aiplatform.AutoMLTextTrainingJob.list(filter = f"display_name={job_display_name}")

if jobs:
    job = jobs[0]
    print(f"Jobs Exists: {jobs[0].display_name}")
else:
    job = aiplatform.AutoMLTextTrainingJob(
    display_name=job_display_name,
    prediction_type="classification",
    multi_label=False,
    )
    print(f"job Created: {job.display_name}")
    
print(f'Review the job in the Console:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/jobs/{jobs[0].resource_name}?project={PROJECT_ID}')

```

print(job)

# Predictions

## Online Prediction 

```python
content = "I got a high score on my math final!"

response = endpoint.predict(instances=[{"content": content}])

for prediction_ in response.predictions:
    ids = prediction_["ids"]
    display_names = prediction_["displayNames"]
    confidence_scores = prediction_["confidences"]
    for count, id in enumerate(ids):
        print(f"Prediction ID: {id}")
        print(f"Prediction display name: {display_names[count]}")
        print(f"Prediction confidence score: {confidence_scores[count]}")
```

##### Another JSON based  online Prediction call 

```python
petfinder_adoption_prediction = petfinder_adoption_endpoint.predict(
    [
        {
            "Type": "Cat",
            "Age": "3",
            "Breed1": "Tabby",
            "Gender": "Male",
            "Color1": "Black",
            "Color2": "White",
            "MaturitySize": "Small",
            "FurLength": "Short",
            "Vaccinated": "No",
            "Sterilized": "No",
            "Health": "Healthy",
            "Fee": "100",
            "PhotoAmt": "2",
        }
    ]
)
print(petfinder_adoption_prediction)
```

## Batch Prediction

### Define the training job and create one if it doesn't exit 

```python

jobs = aiplatform.BatchPredictionJob.list(filter = f"display_name={batch_display_name}")

if jobs:
    batch_predict_job = jobs[0]
    print(f"Jobs Exists: {jobs[0].display_name}")
else:
    batch_predict_job = model.batch_predict(
    job_display_name=batch_display_name,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=OUTPUTBUCKET,
    machine_type="n1-standard-4",
    starting_replica_count=1,
    max_replica_count=1,
    sync=False,
)

```



# Delete Resources

```python
# You must delete any `Model` resources deployed to the `Endpoint` resource 
# before deleting the `Endpoint` resource.

delete_training_job = True
delete_model = True
delete_endpoint = True

# Warning: Setting this to true will delete everything in your bucket
delete_bucket = False

# Delete the training job
petfinder_adoption_job.delete()

# Delete the model
petfinder_adoption_model.delete()

# Delete the endpoint
petfinder_adoption_endpoint.delete()

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil -m rm -r $BUCKET_NAME

```